In [6]:
from tenacity import retry, stop_after_attempt 
import requests
from bs4 import BeautifulSoup
import pandas as pd 


In [7]:
base_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page={}'

@retry(stop=stop_after_attempt(3))
def get_html(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    return soup

all_data = []
max_page = 40



In [8]:
for page in range(1, max_page + 1):
    
    url = base_url.format(page)
    
    soup = get_html(url)
    
    items = soup.find_all("div",{"class":"cassetteitem"})
    print("page", page, "items", len(items))
    
    for item in items:
        stations = item.findAll("div",{"class":"cassetteitem_detail-text"})
        
        for station in stations:
            base_data = {}
            
            base_data["名称"] = item.find("div", {"class":"cassetteitem_content-title"}).getText().strip()
            base_data["カテゴリ"] = item.find("div", {"class":"cassetteitem_content-label"}).getText().strip()
            base_data["住所"] = item.find("li", {"class":"cassetteitem_detail-col1"}).getText().strip()
            base_data["最寄り駅"] = station.getText().strip()
            base_data["築年数"] = item.find("li", {"class":"cassetteitem_detail-col3"}).findAll("div")[0].getText().strip()
            base_data["構造"] = item.find("li", {"class":"cassetteitem_detail-col3"}).findAll("div")[1].getText().strip()
        
            tbodys = item.find("table", {"class":"cassetteitem_other"}).findAll("tbody")
            
            for tbody in tbodys:
                data = base_data.copy()
                
                data["階数"] = tbody.findAll("td")[2].getText().strip()
    
                data["賃料"] = tbody.findAll("td")[3].findAll("li")[0].getText().strip()
                data["管理費"] = tbody.findAll("td")[3].findAll("li")[1].getText().strip()
                
                data["敷金"] = tbody.findAll("td")[4].findAll("li")[0].getText().strip()
                data["礼金"] = tbody.findAll("td")[4].findAll("li")[1].getText().strip()
                
                data["間取り"] = tbody.findAll("td")[5].findAll("li")[0].getText().strip()
                data["専有面積"] = tbody.findAll("td")[5].findAll("li")[1].getText().strip()
            
                all_data.append(data)  

page 1 items 30
page 2 items 30
page 3 items 30
page 4 items 30
page 5 items 30
page 6 items 30
page 7 items 30
page 8 items 30
page 9 items 30
page 10 items 30
page 11 items 30
page 12 items 30
page 13 items 30
page 14 items 30
page 15 items 30
page 16 items 30
page 17 items 30
page 18 items 30
page 19 items 30
page 20 items 30
page 21 items 30
page 22 items 30
page 23 items 30
page 24 items 30
page 25 items 30
page 26 items 30
page 27 items 30
page 28 items 30
page 29 items 30
page 30 items 30
page 31 items 30
page 32 items 30
page 33 items 19
page 34 items 0
page 35 items 0
page 36 items 0
page 37 items 0
page 38 items 0
page 39 items 0
page 40 items 0


In [9]:
# convert to dataframe
df = pd.DataFrame(all_data)
df

,名称,カテゴリ,住所,最寄り駅,築年数,構造,階数,賃料,管理費,敷金,礼金,間取り,専有面積
0,ブライズ神田岩本町,賃貸マンション,東京都千代田区岩本町２,都営新宿線/岩本町駅 歩4分,築17年,15階建,8階,11.9万円,14000円,-,11.9万円,1DK,34.99m2
1,ブライズ神田岩本町,賃貸マンション,東京都千代田区岩本町２,都営新宿線/岩本町駅 歩4分,築17年,15階建,15階,22.4万円,16000円,22.4万円,22.4万円,2LDK,66m2
2,ブライズ神田岩本町,賃貸マンション,東京都千代田区岩本町２,東京メトロ日比谷線/小伝馬町駅 歩4分,築17年,15階建,8階,11.9万円,14000円,-,11.9万円,1DK,34.99m2
3,ブライズ神田岩本町,賃貸マンション,東京都千代田区岩本町２,東京メトロ日比谷線/小伝馬町駅 歩4分,築17年,15階建,15階,22.4万円,16000円,22.4万円,22.4万円,2LDK,66m2
4,ブライズ神田岩本町,賃貸マンション,東京都千代田区岩本町２,ＪＲ山手線/秋葉原駅 歩8分,築17年,15階建,8階,11.9万円,14000円,-,11.9万円,1DK,34.99m2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6946,ＪＲ山手線 秋葉原駅 7階建 築43年,賃貸マンション,東京都千代田区岩本町１,都営新宿線/岩本町駅 歩6分,築43年,7階建,2階,6.5万円,5000円,6.5万円,6.5万円,ワンルーム,17.4m2
6947,ＪＲ山手線 秋葉原駅 7階建 築43年,賃貸マンション,東京都千代田区岩本町１,東京メトロ日比谷線/小伝馬町駅 歩3分,築43年,7階建,2階,6.5万円,5000円,6.5万円,6.5万円,ワンルーム,17.4m2
6948,ＪＲ山手線 神田駅 4階建 築45年,賃貸マンション,東京都千代田区東神田１,ＪＲ山手線/神田駅 歩10分,築45年,4階建,3階,5万円,11000円,-,-,ワンルーム,5m2
6949,ＪＲ山手線 神田駅 4階建 築45年,賃貸マンション,東京都千代田区東神田１,都営浅草線/東日本橋駅 歩5分,築45年,4階建,3階,5万円,11000円,-,-,ワンルーム,5m2


In [10]:
df.to_csv("suumo01.csv", encoding="utf-8")